<a href="https://colab.research.google.com/github/AchrafAsh/gnn-receptive-fields/blob/main/fine_tuning_on_citeseer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODOs:
- [x] sparse edge_index generator (→ not necessarily going from dense to sparse is actually better for the gpu);
- [x] use gpu (but should work with only cpu);
- [x] check the implementation of the model;

## **🚀 Setting up the environment**

In [22]:
import os, sys
import os.path as osp
from google.colab import drive, files
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
try:
    os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
except:
    pass
sys.path.insert(0, nb_path)  # or append(nb_path)

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [2]:
import math
import random
import copy
import time
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import networkx as nx

from itertools import product
from functools import partial
from tqdm.notebook import tqdm
from typing import Dict, List, Tuple
from torch_geometric.utils import degree, to_dense_adj, dense_to_sparse, add_self_loops, to_networkx
from torch_geometric.nn import GCNConv, MessagePassing, Sequential
from torch_sparse import spmm, spspmm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

%matplotlib inline
sns.set_theme(font_scale=1.8)
sns.set_style('whitegrid')

In [3]:
%%capture
!wget https://raw.githubusercontent.com/AchrafAsh/gnn-receptive-fields/main/data.py

from data import load_dataset

## **🎨 Designing the model**

In [4]:
# Parameter initialization
def xavier(tensor):
    if tensor is not None:
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-2)))
        tensor.data.uniform_(-stdv, stdv)


def zeros(tensor):
    if tensor is not None:
        tensor.data.fill_(0)

In [5]:
class OurGCNConv(MessagePassing):
    def __init__(self, num_features:int, in_channels:int, out_channels:int, k:int):
        super().__init__(aggr='add')  # "Add" aggregation
        self.k = k
        self.lin_neb = torch.nn.Linear(num_features, out_channels)
        self.lin_trgt = torch.nn.Linear(in_channels, out_channels)
        
        self.reset_parameters()
        
    def reset_parameters(self):
        xavier(self.lin_neb.weight)
        zeros(self.lin_neb.bias)
        
        xavier(self.lin_trgt.weight)
        zeros(self.lin_trgt.bias)

    def forward(self, x, h, edge_index):
        # x is the input features and has shape [N, num_features]
        # h is the hidden state and has shape [N, in_channels]
        # edge_index has shape [2, E] , E being the number of edges

        # step 1: linearly transform node feature matrices
        x = self.lin_neb(x)
        h = self.lin_trgt(h)

        # step 3-5: start propagating messages
        return self.propagate(next(edge_index), x=x, h=h)

    def message(self, x_j, h_i, edge_index, size):
        # x_j is the input features of the neighbors and has shape [E, out_channels] (has already been multiplied by the weight matrix)

        # step 3: normalize node features
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        out = norm.view(-1, 1) * x_j

        return out + h_i

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels] is the output of self.message()

        # step 5: return new node embeddings
        return aggr_out

In [6]:
# the real deal
class OurModel(torch.nn.Module):
    def __init__(self, num_layers:int, hidden_dim:int, num_features:int, 
                 num_classes:int, propagation_steps:int=2, dropout:float=0.5):
        super().__init__()
        self.propagation_steps = propagation_steps
        self.num_layers = num_layers
        
        self.alpha = torch.nn.Parameter(torch.tensor(0.5), requires_grad=True)
        # Embedding input features
        self.in_mlp = torch.nn.Sequential(
            torch.nn.Linear(in_features=num_features, out_features=hidden_dim),
            torch.nn.ReLU()
        )
        # Convolutional layers
        self.conv_layers = self.create_layers(num_layers=num_layers,
                                              hidden_dim=hidden_dim,
                                              dropout=dropout)
        # Readout function
        self.readout = torch.nn.Sequential(
            torch.nn.Linear(in_features=hidden_dim, out_features=num_classes),
            torch.nn.LogSoftmax(dim=1)
        )

    def create_layers(self, num_layers:int, hidden_dim:int, dropout:float):
        layers = [(OurGCNConv(num_features=hidden_dim, in_channels=hidden_dim, out_channels=hidden_dim, k=0), "x, x, edge_index -> h"),
                (torch.nn.ReLU(inplace=True)),
                (torch.nn.Dropout(p=dropout), "h -> h")]
        
        for k in range(1, num_layers):
            layers += [
                (OurGCNConv(num_features=hidden_dim, in_channels=hidden_dim, out_channels=hidden_dim, k=k), "x, h, edge_index -> h"),
                # (GCNConv(hidden_dim, hidden_dim), "h, edge_index -> h"),
                (torch.nn.ReLU(inplace=True)),
                (torch.nn.Dropout(p=dropout), "h -> h")
            ]
        return Sequential("x, edge_index", layers)

    def reset_parameters(self):
        self.conv_layers.reset_parameters()


    def propagate(self, x, edge_index):
        # add self loops
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        # normalize
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # dense_adj = to_dense_adj(edge_index).squeeze(0)
        # edge_index_pow = torch.eye(G.num_nodes).to(device)
        # prop_repr = edge_index_pow.clone()
        # for _ in range(self.propagation_steps):
        #     edge_index_pow = torch.mm(dense_adj, edge_index_pow)
        #     prop_repr += edge_index_pow
        # return torch.mm(prop_repr, x)
        
        # APPNP propagation scheme
        z = x.clone()
        for _ in range(self.propagation_steps):
            z = spmm(edge_index, norm, x.size(0), x.size(0), z) * (1-self.alpha) + x * self.alpha
        return z

        # another propagation scheme (sum of the powers of A)
        # props = []
        # for _ in range(self.propagation_steps):
        #     x= spmm(edge_index, norm, x.size(0), x.size(0), x)
        #     props.append(x)
        # return sum(props)


    def forward(self, x):
        # compute different depth edge_index
        edge_index_gen = k_hop_neighbors(self.num_layers, G.edge_index)

        embeddings = self.in_mlp(x)
        h = self.conv_layers(embeddings, edge_index_gen)
        out = self.propagate(h, G.edge_index)
        return h, self.readout(out)

## **🧰 Utility functions**

In [7]:
def clamp(x: torch.Tensor):
    if not x.is_coalesced(): x = x.coalesce()

    mask = (x._values() > 0).nonzero().view(-1)
    values = x._values().index_select(0, mask).clamp(0, 1)
    indices = x._indices().index_select(1, mask)

    return torch.sparse_coo_tensor(indices, values, x.shape).coalesce()

def sparse_hop_neighbors(k:int, edge_index: torch.Tensor, num_nodes:int):
    # transform edge_index into a sparse tensor
    yield edge_index, edge_index

    if k > 1:
        sparse_edge_index = torch.sparse_coo_tensor(edge_index, torch.ones(edge_index.size(1), device=device), (num_nodes, num_nodes))
        cum_neighbors = neighbors = pow_A = sparse_edge_index.clone()

    for _ in range(1, k):
        pow_A = clamp(torch.sparse.mm(sparse_edge_index, pow_A))
        neighbors = clamp(pow_A - cum_neighbors)
        cum_neighbors = (cum_neighbors + neighbors).coalesce()
        
        yield neighbors.indices()

In [8]:
def k_hop_neighbors(k: int, edge_index: torch.Tensor):
    yield edge_index
    dense_adj = to_dense_adj(edge_index).squeeze(0)
    cum_adj = adj_pow = dense_adj.clone()
    for l in range(1, k):
        adj_pow = torch.mm(dense_adj, adj_pow)
        k_hop = torch.where(
            torch.where(adj_pow > 0, 1, 0) - cum_adj > 0,
            1,
            0
        )
        cum_adj += k_hop
        yield dense_to_sparse(k_hop)[0] # return only the edge indices

In [18]:
# count model parameters
def count_parameters(model: torch.nn.Module):
    total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_parameters

In [27]:
def make(config):
    # Make the model
    model = OurModel(num_layers=config['num_layers'],
                     hidden_dim=config['hidden_dim'],
                     num_features=amazon.num_features,
                     num_classes=amazon.num_classes,
                     propagation_steps=config['propagation_steps'],
                     dropout=config['dropout']).to(device)

    # Make the loss and optimizer
    criterion = torch.nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=config['learning_rate'],
                                 weight_decay=config['weight_decay'])
    
    return model, criterion, optimizer

In [19]:
def train(model, criterion, optimizer, config):
    outputs = []
    
    for _ in range(config['runs']):
        for epoch in range(config['epochs']):
            loss = train_step(model, optimizer, criterion)
            
            # test the model
            outs = test(model, criterion)
                        # metrics=[('mad', mad_value),
                        #         ('mad_gap', partial(mad_gap_value,dense_adj)),
                        #         ('mad_sp', shortest_path_mad)])
            outs['epoch'] = epoch
            outs['id'] = config['id']
            outs['hidden_dim'] = config['hidden_dim']
            outs['weight_decay'] = config['weight_decay']
            outs['num_layers'] = config['num_layers']
            outs['learning_rate'] = config['learning_rate']
            outs['dropout'] = config['dropout']
            outs['propagation_steps'] = config['propagation_steps']
            outs['total_parameters'] = config['total_parameters']
            outputs.append(outs)

    return pd.DataFrame(outputs)

In [12]:
def train_step(model, optimizer, criterion):
    """Performs one training step
    """
    model.train()
    
    # Forward pass
    _, out = model(G.x)
    loss = criterion(out[G.train_mask], G.y[G.train_mask])
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss

In [13]:
def test(model, criterion, metrics=[]):
    """
    Metrics is a list of tuple ('metric_name', metric_func) where the metric 
    function takes the last representation matrix and returns a scalar.
    """
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        h, logits = model(G.x)

    outs = {}
    h = h.detach().cpu()
    for (name, metric) in metrics:
        outs[name] = metric(h)

    for key in ['train', 'val', 'test']:
        mask = G[f'{key}_mask']
        loss = criterion(logits[mask], G.y[mask]).item()
        pred = logits[mask].max(1)[1]
        acc = pred.eq(G.y[mask]).sum().item() / mask.sum().item()

        outs[f'{key}_loss'] = loss
        outs[f'{key}_acc'] = acc
    
    return outs

In [14]:
def model_pipeline(config):
    # create the model
    model, criterion, optimizer = make(config)
    config['total_parameters'] = count_parameters(model)

    # train the model for different parameters
    logs = train(model, criterion, optimizer, config)

    # repr = tsne_plot(model, all_edge_index, title="Last hidden representations")

    return model, logs, config

## **🕸 Citeseer**

In [15]:
%%capture
path = osp.join(os.getcwd(), 'data')
citeseer = load_dataset(path, 'CiteSeer')
G = citeseer[0].to(device) # only graph of the dataset

## **🔍Fine tuning**

In [16]:
grid = dict(
    num_layers=[1,2],
    hidden_dim=[16],
    propagation_steps=[6,7,8,9],
    learning_rate=np.logspace(-4, -1, 5),
    weight_decay=np.logspace(-5, 0, 5),
    dropout=np.linspace(0.4,0.6, 5)
)

In [20]:
id = 0
all_logs=None
best_acc = 0
hyperparameters = list(product(*grid.values()))

for num_layers, hidden_dim, propagation_steps, learning_rate, weight_decay, dropout in tqdm(hyperparameters):
    model, logs, hyperparameters = model_pipeline({
        'id':id,
        'num_layers':num_layers,
        'hidden_dim':hidden_dim,
        'propagation_steps':propagation_steps,
        'learning_rate':learning_rate,
        'weight_decay':weight_decay,
        'dropout':dropout,
        'epochs':400,
        'runs':1
    })
    test_acc = logs.query('epoch==199')['test_acc'].iloc[0]
    if test_acc >= best_acc:
        best_acc = test_acc
        print(f"num_layers: {num_layers} \
            | smoothing: {propagation_steps} \
            | dim: {hidden_dim} \
            | test_acc: {test_acc} \n")
    
    if id == 0:
        all_logs = logs
    else:
        all_logs = pd.concat([all_logs, logs], ignore_index=True)
    id += 1

  0%|          | 0/1000 [00:00<?, ?it/s]

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.442 

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.527 

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.543 

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.577 

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.611 

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.639 

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.641 

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.646 

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.649 

num_layers: 1             | smoothing: 6             | dim: 16             | test_acc: 0.658 

num_layers: 1             | smoothing: 6          

In [21]:
all_logs.query('epoch == 199 & train_acc == 1.0 & test_acc > 0.7 & val_acc > 0.7').sort_values(by=["test_acc", "val_acc"], ascending=False)

,train_loss,train_acc,val_loss,val_acc,test_loss,test_acc,epoch,id,hidden_dim,weight_decay,num_layers,learning_rate,dropout,propagation_steps,total_parameters


In [30]:
all_logs = pd.read_csv('tuning_citeseer.csv')

In [31]:
all_logs.query('epoch > 100 & train_acc > 0.5 & test_acc > 0.4').to_csv('./tuning_citeseer_small.csv', index=False)
files.download('tuning_citeseer_small.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **📷 Amazon Photos**

In [24]:
%%capture
path = osp.join(os.getcwd(), 'data')
amazon = load_dataset(path, 'Amazon', params=dict(goods="Photo"))
G = amazon[0].to(device)

## **🔍Fine tuning**

In [25]:
grid = dict(
    num_layers=[1,2],
    hidden_dim=[16,24],
    propagation_steps=[5,7,9],
    learning_rate=np.logspace(-4, -1, 5),
    weight_decay=np.logspace(-5, 0, 5),
    dropout=np.linspace(0.4,0.6, 5)
)

In [28]:
id = 0
all_logs=None
best_acc = 0
hyperparameters = list(product(*grid.values()))

for num_layers, hidden_dim, propagation_steps, learning_rate, weight_decay, dropout in tqdm(hyperparameters):
    model, logs, hyperparameters = model_pipeline({
        'id':id,
        'num_layers':num_layers,
        'hidden_dim':hidden_dim,
        'propagation_steps':propagation_steps,
        'learning_rate':learning_rate,
        'weight_decay':weight_decay,
        'dropout':dropout,
        'epochs':400,
        'runs':1
    })
    test_acc = logs.query('epoch==199')['test_acc'].iloc[0]
    if test_acc >= best_acc:
        best_acc = test_acc
        print(f"num_layers: {num_layers} \
            | smoothing: {propagation_steps} \
            | dim: {hidden_dim} \
            | test_acc: {test_acc} \n")
    
    if id == 0:
        all_logs = logs
    else:
        all_logs = pd.concat([all_logs, logs], ignore_index=True)
    id += 1

  0%|          | 0/1500 [00:00<?, ?it/s]

AttributeError: ignored

In [ ]:
all_logs.query('epoch == 199 & train_acc == 1.0 & test_acc > 0.7 & val_acc > 0.7').sort_values(by=["test_acc", "val_acc"], ascending=False)